<a href="https://colab.research.google.com/github/Stanfording/small_GAN_testing/blob/main/64_cele_small_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Get the preprocessed celebHD data from google drive

#!mv "/content/drive/MyDrive/Proressive GAN/proCeleba.zip" "content"

!unzip "/content/drive/MyDrive/Proressive GAN/proCeleba.zip" "/content"

Archive:  /content/drive/MyDrive/Proressive GAN/proCeleba.zip
caution: filename not matched:  /content


In [ ]:
#!zip "/content/proCeleba" "/content/drive/MyDrive/Proressive GAN/"

updating: content/drive/MyDrive/Proressive GAN/ (stored 0%)


In [ ]:
#!mv "/content/proCeleba.zip" "/content/drive/MyDrive/Proressive GAN/"

In [ ]:
# #!/usr/bin/env python3
# # -*- coding: utf-8 -*-
# """
# Created on Fri May  6 10:34:58 2022

# @author: Stanford
# """


'''Change the folder into tfrecord folder using tensorflow2'''



import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow as tf
import os
import torch as t
from torchvision import datasets, transforms, utils
from torch.utils.data import Dataset, DataLoader
from skimage import io


class Get_No_Classes_Img_Dataset(Dataset):
    
    def __init__(self, folder_dir, transform = None):
        self.folder_dir = os.path.join(folder_dir)
        self.transform = transform
        self.image_list = os.listdir(self.folder_dir)
        
    def __len__(self):
        return len(os.listdir(self.folder_dir))
    
    def __getitem__(self, index):
        
        image_name = self.image_list[index]
        
        image_dir = os.path.join(self.folder_dir, image_name)
        
        image = io.imread(image_dir)
        
        return image
        
        
        

def conver_pic_dif_reso(path_of_1024_folder):
    
    '''
    path_of_1024_folder: 
        string or os.path
        folder to transfer to tfrecord
    
    
    
    1. Get a photo
    2. create reso dir
    3. get all 1024 photos
    4. transfer them to lower reso and feed them to according files
    '''
    
    
    ''' Get a list of directories for all photos '''
    folder = os.path.join(path_of_1024_folder)
    photo_name_list = os.listdir(folder)
    photo_dir = []
    
    for name in photo_name_list:
        aphoto_dir = os.path.join(folder, name)
        photo_dir.append(aphoto_dir)
        
    dir_to_name = {}
    name_to_dir = {}
    for i in range(len(photo_name_list)):
        dir_to_name[photo_dir[i]] = photo_name_list[i]
        name_to_dir[photo_name_list[i]] = photo_dir[i]
    
    ''' 1. Get a photo'''  
    
    a_photo = mpimg.imread(photo_dir[0])
    w, h, c = a_photo.shape
    
    ''' 2. create reso dir'''
    
    num_of_file = int(np.log2(w))
    
    upper_folder = ""
    if str(folder)[-1] != "/":
        upper_folder = os.path.dirname(folder)
    else:
        upper_folder = os.path.dirname(os.path.dirname(folder))
    
    reso_dir = []
    
    for i in range(num_of_file - 1, 2, -1):
        new_folder_dir = os.path.join(upper_folder, f"{2 ** i}")
        #os.mkdir(new_folder_dir)
        reso_dir.append(new_folder_dir)
    
    ''' 3. define a loop function that take a 1024 photo, then reshape and put it into according reso folder '''
    
    for one_photo_dir in photo_dir:
        ''' 1. get a photo'''
        try:
            a_photo = io.imread(one_photo_dir)
        except:
            print(one_photo_dir)
            continue
        a_photo_name = dir_to_name[one_photo_dir]
        
        for i in range(num_of_file-1, 2, -1):
            reso = 2 ** i
            resizeFunc = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((reso, reso))
                ])
            lowerReso = resizeFunc(a_photo)
            according_dir = reso_dir[9-i]
            according_dir_img_name = os.path.join(according_dir, a_photo_name)
            lowerReso.save(according_dir_img_name)
            
            
    
    
    ''' define a transform function'''
    

    
    
    

    return


folder = "/Users/Stanford/Desktop/Let's Go！/AI/GAN/proGAN/Self_modified ProGAN/proCeleba/1024/"

conver_pic_dif_reso(folder)



























In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
